In [11]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import plot_confusion_matrix

In [12]:
df = pd.read_csv('all_4000_posts.csv')

In [13]:
df.drop(columns="Unnamed: 0", inplace=True)

In [14]:
df['selftext'] = df['selftext'].fillna('n/a')


In [15]:
df['title_and_st'] = df['title'] + df['selftext']

In [16]:
df.duplicated(subset=None, keep='first').sum()

0

In [17]:
df['subreddit'] = df['subreddit'].map({'Liberal':1, "Conservative":0})

In [18]:
X = df['title_and_st']
y = df['subreddit']